In [1]:
# -*- coding: utf-8 -*-
import os
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as pl
from joblib import dump, load
# import pickle

# GLM
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.genmod.families import family
from statsmodels.stats.multitest import multipletests
 
# Modelling
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

from sklearn.linear_model import Ridge, Lasso, RidgeCV, LassoCV, ElasticNet, LogisticRegression
from sklearn.svm import SVR, SVC
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, GradientBoostingClassifier, StackingRegressor, StackingClassifier
from xgboost import XGBRegressor, XGBClassifier
import xgboost
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from mlxtend.regressor import StackingCVRegressor
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

# Mertrics
from sklearn.metrics import make_scorer, mean_squared_error, root_mean_squared_error, \
    accuracy_score, confusion_matrix, precision_score, roc_curve, recall_score, \
        precision_recall_curve, precision_recall_fscore_support, roc_auc_score, \
            ConfusionMatrixDisplay, r2_score, classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split, cross_val_score
from scipy.stats import randint

os.chdir('/Users/jhou2/Documents/GitHub/PertussisVaccine_Prediction/')

In [2]:
# Load data
subject_specimen_df = pd.read_csv('data/output/subject_specimen.csv')
Ab_titer_df = pd.read_csv('data/output/Ab_titer.csv')
RNAseq_df = pd.read_csv('data/output/RNAseq.csv')
Cell_Freq_df = pd.read_csv('data/output/Cell_Freq.csv')

In [3]:
# drop unnecessary columns from each dataframe as sample will be annotated later after merging
excluede_columns = ['subject_id', 'dataset', 'timepoint', 'infancy_vac', 'biological_sex', 'date_of_boost', 'race', 'age', 'age_at_boost']

Ab_titer_df = Ab_titer_df.drop(columns = excluede_columns)
RNAseq_df = RNAseq_df.drop(columns = excluede_columns)
Cell_Freq_df = Cell_Freq_df.drop(columns = excluede_columns)

In [4]:
# Feature engineering: remove non-specific features
remove_features_ab = Ab_titer_df.columns.str.contains('TT|DT|OVA')
Ab_titer_df = Ab_titer_df.loc[:, ~remove_features_ab]

remove_features_cellfreq = Cell_Freq_df.columns.str.contains('CD3CD19neg|CD3.Tcells|B.cells..CD19.CD3.CD14.CD56..|B.cells..CD19.CD20.CD3.CD14.CD56..|CD56.CD3.T.cells|CD4.CD8..T.cells|CD4.CD8..T.cells.1|NK.cells..CD3.CD19.CD56..|CD3.CD19.CD56..cells|non.pDCs|CD3.CD19.CD56.CD14.CD16.CD123.CD11c.HLA.DR.cells|Lineage.negative.cells..CD3.CD19.CD56.HLA.DR.CD123.CD66b..')
Cell_Freq_df = Cell_Freq_df.loc[:, ~remove_features_cellfreq]

# Compute the most variable genes from RNAseq data
gene_matrix = RNAseq_df.drop(columns=['specimen_id'])
gene_variances = gene_matrix.var(axis=0)
top_n = 1000 
top_genes = gene_variances.sort_values(ascending=False).head(top_n).index
filtered_gene_matrix = gene_matrix[top_genes]
filtered_RNAseq_df = pd.concat([RNAseq_df['specimen_id'], filtered_gene_matrix], axis=1)

In [5]:
# Merge all dataframes on 'specimen_id'
all_data_df = pd.merge(Ab_titer_df, 
                       Cell_Freq_df, 
                       on='specimen_id', how='outer')
all_data_df = pd.merge(all_data_df, 
                       filtered_RNAseq_df, 
                       on='specimen_id', how='outer')
print(all_data_df.shape)

# annotate the specimen_id with subject_id and age
all_data_df = pd.merge(all_data_df, 
                       subject_specimen_df, 
                       on='specimen_id', how='inner')
print(all_data_df.shape)

# remove 2023 dataset as it missing target variable
all_data_df = all_data_df[all_data_df['dataset'] != '2023_dataset']
print(all_data_df.shape)

(883, 1047)
(883, 1056)
(721, 1056)


In [6]:
# Extract the feature and target variables from the dataframe
# Feature variables
day0_df = all_data_df[all_data_df['timepoint'] == 0].drop(columns=['specimen_id','timepoint', 'dataset', 'date_of_boost', 'race', 'age_at_boost'])

# Predict target
Ab_titer_day14 = all_data_df[all_data_df['timepoint']== 14][['subject_id', 'IgG_PT']].rename(columns={'IgG_PT': 'IgG_PT_day14'})
Monocytes_day1 = all_data_df[all_data_df['timepoint']== 1][['subject_id', 'Monocytes']].rename(columns={'Monocytes': 'Monocytes_day1'})
CCL3_day3 = all_data_df[all_data_df['timepoint']== 3][['subject_id', 'ENSG00000277632.1']].rename(columns={'ENSG00000277632.1': 'CCL3_day3'})

In [7]:
# Combine Day0 features with target variables
# and compute ratios
final_df = pd.merge(day0_df, Ab_titer_day14, on='subject_id', how='inner')
final_df['IgG_PT_ratio'] = final_df['IgG_PT_day14'] / final_df['IgG_PT']

final_df = pd.merge(final_df, Monocytes_day1, on='subject_id', how='inner')
final_df['Mono_ratio'] = final_df['Monocytes_day1'] / final_df['Monocytes']

final_df = pd.merge(final_df, CCL3_day3, on='subject_id', how='inner')
final_df['CCL3_ratio'] = final_df['CCL3_day3'] / final_df['ENSG00000277632.1']

# clean up final_df.columns
columns_to_exclude = ['subject_id']
final_df = final_df.drop(columns=columns_to_exclude)

In [8]:
# One-hot encode categorical variables
encoder = OneHotEncoder(sparse_output=False)
columns_to_onehot_encode = ['infancy_vac', 'biological_sex']
one_hot_encoded = encoder.fit_transform(final_df[columns_to_onehot_encode])
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(columns_to_onehot_encode))

# Concatenate one-hot encoded features with the original DataFrame
final_df_encoded = pd.concat([final_df, one_hot_df], axis=1)
final_df_encoded = final_df_encoded.drop(columns_to_onehot_encode, axis=1)

In [9]:
def data_preprocessing(predictor):
    # Define predictor and drop NaN values in the target variable
    temp = final_df_encoded.dropna(subset=[predictor])

    # Define columns to exclude from features
    no_day0_columns = ['IgG_PT_day14', 'Monocytes_day1', 'CCL3_day3', 'IgG_PT_ratio', 'Mono_ratio', 'CCL3_ratio']

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(temp.drop(columns=no_day0_columns), 
                                                        temp[predictor], 
                                                        test_size=0.3, 
                                                        random_state=42)
    
    # Impute missing values using KNNImputer
    columns_to_exclude = ['age', 'infancy_vac_aP', 'infancy_vac_wP', 'biological_sex_Female', 'biological_sex_Male']
    columns_to_impute = [col for col in X_train.columns if col not in columns_to_exclude]

    # Initialize and fit KNNImputer on training data only
    imputer = KNNImputer(n_neighbors=3)
    X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train[columns_to_impute]), 
                                columns=columns_to_impute, 
                                index=X_train.index)
    X_train_imputed[columns_to_exclude] = X_train[columns_to_exclude]

    # Transform test data using the fitted imputer
    X_test_imputed = pd.DataFrame(
        imputer.transform(X_test[columns_to_impute]),
        columns=columns_to_impute, 
        index=X_test.index
    )
    X_test_imputed[columns_to_exclude] = X_test[columns_to_exclude]

    # Standardize the data
    # Fit the standardization scaler onto the training data (only for columns to impute)
    scaler = StandardScaler().fit(X_train_imputed[columns_to_impute])

    # Transform the training data
    X_train_scaled = pd.DataFrame(
        scaler.transform(X_train_imputed[columns_to_impute]),
        columns=columns_to_impute,
        index=X_train_imputed.index
    )
    X_train_scaled[columns_to_exclude] = X_train_imputed[columns_to_exclude]

    # Use the same scaler to transform the testing set
    X_test_scaled = pd.DataFrame(
        scaler.transform(X_test_imputed[columns_to_impute]),
        columns=columns_to_impute,
        index=X_test_imputed.index
    )
    X_test_scaled[columns_to_exclude] = X_test_imputed[columns_to_exclude]
    
    return X_train_scaled, X_test_scaled, y_train, y_test

In [10]:
# Test some basic models, aim to 4 basic models first
def basic_models(X_train, y_train, X_test, y_test):
    # model 1: Lasso Regression
    lasso_basic_model = Lasso(alpha=1, random_state=42)
    lasso_basic_model.fit(X_train, y_train)
    y_pred_lasso = lasso_basic_model.predict(X_test)
    test_score_lasso = lasso_basic_model.score(X_test, y_test)
    rmse_lasso = root_mean_squared_error(y_test, y_pred_lasso)

    # model 2: Ridge Regression
    ridge_basic_model = Ridge(alpha=1, random_state=42)
    ridge_basic_model.fit(X_train, y_train)
    y_pred_ridge = ridge_basic_model.predict(X_test)
    test_score_ridge = ridge_basic_model.score(X_test, y_test)
    rmse_ridge = root_mean_squared_error(y_test, y_pred_ridge)

    # model 3: Random Forest
    RF_basic_model = RandomForestRegressor(n_estimators=100, random_state=42)
    RF_basic_model.fit(X_train, y_train)
    y_pred_RF = RF_basic_model.predict(X_test)
    test_score_RF = RF_basic_model.score(X_test, y_test)
    rmse_RF = root_mean_squared_error(y_test, y_pred_RF)

    # model 4: XGBoost
    xgb_basic_model = XGBRegressor(n_estimators=1000, learning_rate=0.01, random_state=42)
    xgb_basic_model.fit(X_train, y_train)
    y_pred_xgb = xgb_basic_model.predict(X_test)
    test_score_xgb = xgb_basic_model.score(X_test, y_test)
    rmse_xgb = root_mean_squared_error(y_test, y_pred_xgb)

    # model 5: Gradient Boosting
    gb_basic_model = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.01, random_state=42)
    gb_basic_model.fit(X_train, y_train)
    y_pred_gb = gb_basic_model.predict(X_test)
    test_score_gb = gb_basic_model.score(X_test, y_test)
    rmse_gb = root_mean_squared_error(y_test, y_pred_gb)

    # model 6: SVM
    svm_basic_model = SVR(kernel='linear')
    svm_basic_model.fit(X_train, y_train)
    y_pred_svm = svm_basic_model.predict(X_test)
    test_score_svm = svm_basic_model.score(X_test, y_test)
    rmse_svm = root_mean_squared_error(y_test, y_pred_svm)

    return pd.DataFrame({
        'Model': ['Lasso', 'Ridge','Random Forest', 'XGBoost', 'Gradient Boosting','SVM'],
        'Test Score': [test_score_lasso, test_score_ridge, test_score_RF, test_score_xgb, test_score_gb, test_score_svm],
        'RMSE': [rmse_lasso, rmse_ridge, rmse_RF, rmse_xgb, rmse_gb, rmse_svm]
    })


In [11]:
# For IgG_PT target
X_train_stan, X_test_stan, y_train, y_test = data_preprocessing('IgG_PT_day14')
basic_models(X_train_stan, y_train, X_test_stan, y_test)

,Model,Test Score,RMSE
0,Lasso,0.047395,3.666203
1,Ridge,-1.834593,6.324196
2,Random Forest,0.008539,3.740226
3,XGBoost,-0.332665,4.336311
4,Gradient Boosting,-0.197543,4.110602
5,SVM,-0.822408,5.070874


In [12]:
# For IgG_PT ratio
X_train_stan, X_test_stan, y_train, y_test = data_preprocessing('IgG_PT_ratio')
basic_models(X_train_stan, y_train, X_test_stan, y_test)

,Model,Test Score,RMSE
0,Lasso,-1.867490,23.091940
1,Ridge,-10.116304,45.466311
2,Random Forest,0.287184,11.513256
3,XGBoost,0.085940,13.037574
4,Gradient Boosting,0.091332,12.999064
5,SVM,-0.773089,18.158282


In [13]:
# For CCL3 day3
X_train_stan, X_test_stan, y_train, y_test = data_preprocessing('CCL3_day3')
basic_models(X_train_stan, y_train, X_test_stan, y_test)

,Model,Test Score,RMSE
0,Lasso,-0.000006,1.647291
1,Ridge,0.243387,1.432868
2,Random Forest,0.313595,1.364769
3,XGBoost,0.186429,1.485822
4,Gradient Boosting,0.270943,1.406533
5,SVM,0.253144,1.423599


In [14]:
# For CCL3 ratio
X_train_stan, X_test_stan, y_train, y_test = data_preprocessing('CCL3_ratio')
basic_models(X_train_stan, y_train, X_test_stan, y_test)

,Model,Test Score,RMSE
0,Lasso,-0.004734,0.323586
1,Ridge,0.156591,0.296472
2,Random Forest,0.022976,0.319093
3,XGBoost,-0.062149,0.332703
4,Gradient Boosting,0.062500,0.312572
5,SVM,0.041830,0.315999


In [15]:
# For Monocytes day1
X_train_stan, X_test_stan, y_train, y_test = data_preprocessing('Monocytes_day1')
basic_models(X_train_stan, y_train, X_test_stan, y_test)

,Model,Test Score,RMSE
0,Lasso,-0.055476,0.528084
1,Ridge,0.220254,0.453895
2,Random Forest,0.265592,0.440501
3,XGBoost,0.150643,0.473722
4,Gradient Boosting,0.247148,0.445998
5,SVM,0.240258,0.448035


In [16]:
# For Mono ratio
X_train_stan, X_test_stan, y_train, y_test = data_preprocessing('Mono_ratio')
basic_models(X_train_stan, y_train, X_test_stan, y_test)

,Model,Test Score,RMSE
0,Lasso,-0.040330,0.341683
1,Ridge,-0.736725,0.441472
2,Random Forest,-0.015348,0.337556
3,XGBoost,-0.184165,0.364539
4,Gradient Boosting,-0.347262,0.388834
5,SVM,-0.612739,0.425422


In [ ]:
# For CCL3_day3 prediction target, Random Forest has the best performance on the basic models
# Select this as example for hyperparameter tuning with Bayesian optimization
n_iter = 100

search_space_rf = {
    'n_estimators': (100, 1000),
    'max_depth': (3, 30),
    'min_samples_split': (2, 10),
    'min_samples_leaf': (1, 10),
    'max_features': ['sqrt', 'log2']
}

# Use negative RMSE as the scoring metric
neg_rmse = make_scorer(root_mean_squared_error, greater_is_better=False)

rf_reg_grid = BayesSearchCV(estimator=RandomForestRegressor(random_state=42),
                              search_spaces=search_space_rf,
                              n_iter=n_iter,
                              cv=5,
                              n_jobs=-1,
                              scoring=neg_rmse,
                              random_state=123)

# For CCL3 day3
X_train_stan, X_test_stan, y_train, y_test = data_preprocessing('CCL3_day3')

rf_reg_grid.fit(X_train_stan, y_train)

print("Best Random Forest Classifier Parameters:", rf_reg_grid.best_params_)

In [ ]:
best_rf = rf_reg_grid.best_estimator_
y_pred = best_rf.predict(X_test_stan)

r2 = r2_score(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
n = X_test_stan.shape[0]
p = X_test_stan.shape[1]
adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)

print(f"R-squared: {r2:.4f}")
print(f"Adjusted R-squared: {adjusted_r2:.4f}")
print(f"RMSE: {rmse:.4f}")

R-squared: 0.1718
Adjusted R-squared: 1.0277
RMSE: 0.1556
